<a href="https://colab.research.google.com/github/zapatos24/luria_vote_data_closeout/blob/jeremy-wip/elaine_luria_vote_data_closeout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Elaine Luria Voter Data Closeout**

In [1]:
# !pip install gspread-pandas
!pip install --upgrade gspread

Requirement already up-to-date: gspread in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [2]:
import pandas as pd
import requests
import json
import time

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [4]:
#Parse all counties associated with VA 02
house_district = '02'
counties = []
path = "https://results.elections.virginia.gov/vaelections/2020%20November%20General/Json/Member_House_of_Representatives_({}).json".format(house_district)

r = requests.get(path)

county_data = json.loads(r.text)
for locality in county_data['Localities']:
    counties.append(locality['Locality']['LocalityName'])

print(counties)

['ACCOMACK COUNTY', 'HAMPTON CITY', 'JAMES CITY COUNTY', 'NEWPORT NEWS CITY', 'NORFOLK CITY', 'NORTHAMPTON COUNTY', 'POQUOSON CITY', 'VIRGINIA BEACH CITY', 'WILLIAMSBURG CITY', 'YORK COUNTY']


In [5]:
#iterate through counties in district and pull data for each precinct into list of dictionaries
precinct_house_list = []
precinct_pres_list = []

#names on the ballot for each candidate
d_house_name = 'Elaine G. Luria'
r_house_name = 'Scott W. Taylor'
d_pres_name = 'Joseph R. Biden'
r_pres_name = 'Donald J. Trump'

#create new list of counties that replace a space with an underscore
counties_no_space = [c.replace(' ', '_') for c in counties]

site_source = 'https://results.elections.virginia.gov/'

#iterate through counties and parse precinct level vote data
for county in counties_no_space:
    house_county_path = 'vaelections/2020%20November%20General/Json/Locality/{}/Member_House_of_Representatives_({}).json'.format(county, house_district)
    pres_county_path = 'vaelections/2020%20November%20General/Json/Locality/{}/President_and_Vice_President.json'.format(county)
    
    house_path = site_source + house_county_path
    pres_path = site_source + pres_county_path

    #create dict for house precinct results
    r_house = requests.get(house_path)
    precinct_house_data = json.loads(r_house.text)

    #create dict for pres precinct results
    r_pres = requests.get(pres_path)
    precinct_pres_data = json.loads(r_pres.text)

    #iterate through house data for each precinct
    for p in precinct_house_data['Precincts']:
        p_dict = {}
        p_dict['Precinct'] = p['PrecinctName'][:-5]
        p_dict['State Senate District'] = ''
        p_dict['House District'] = ''
        p_dict['County/Locality'] = county

        #for each list of candidates, add data for corresponding house candidate
        for i in range(len(p['Candidates'])):
            if p['Candidates'][i]['BallotName'] == r_house_name:
                p_dict['Taylor Votes'] = p['Candidates'][i]['Votes']
                p_dict['Taylor Percentage'] = p['Candidates'][i]['Percentage']
            elif p['Candidates'][i]['BallotName'] == d_house_name:
                p_dict['Luria Votes'] = p['Candidates'][i]['Votes']
                p_dict['Luria Percentage'] = p['Candidates'][i]['Percentage']

        #append info for precinct to list of precinct house data
        precinct_house_list.append(p_dict)

    #iterate through president data for each precinct
    for p in precinct_pres_data['Precincts']:
        p_dict = {}

        #only parse data for those in 
        if p['PrecinctName'][-4:] == '(02)':
            p_dict['Precinct'] = p['PrecinctName'][:-5]
            p_dict['County/Locality'] = county

        #for each list of candidates, add data for corresponding pres candidate
            for i in range(len(p['Candidates'])):
                if p['Candidates'][i]['BallotName'] == r_pres_name:
                    p_dict['Trump Votes'] = p['Candidates'][i]['Votes']
                    p_dict['Trump Percentage'] = p['Candidates'][i]['Percentage']
                elif p['Candidates'][i]['BallotName'] == d_pres_name:
                    p_dict['Biden Votes'] = p['Candidates'][i]['Votes']
                    p_dict['Biden Percentage'] = p['Candidates'][i]['Percentage']

            #append info for precinct to list of precinct pres data
            precinct_pres_list.append(p_dict)

    #short sleep to not overtax government servers
    time.sleep(.5)

In [6]:
#Convert lists of dictionaries into pandas dataframes and join on County and Precinct
house_df = pd.DataFrame(precinct_house_list)
pres_df = pd.DataFrame(precinct_pres_list)

main_df = house_df.merge(pres_df, how='left', on=['Precinct', 'County/Locality'])

In [7]:
export_cols = ["Precinct",
               "State Senate District",	
               "House District",
               "County/Locality",
               "Taylor Votes",
               "Taylor Percentage",
               "Luria Votes",
               "Luria Percentage",
               "Trump Votes",
               "Trump Percentage",
               "Biden Votes",
               "Biden Percentage"]

#reorder df for preferred column order
export_df = main_df[export_cols]

In [8]:
#Update df values into Google Sheet
sh = gc.open("Voter Data Close Out Project 2020")
wks = sh.worksheet("VA02 Results")

cell_range = 'A2:L{}'.format(str(len(export_df)+1))

wks.update('A2:L1000', export_df.values.tolist())

{'spreadsheetId': '1TljHbNTUYeXcPAC4U0OtxrrcUMa_ZpO3qq4wZ39ysgw',
 'updatedCells': 2316,
 'updatedColumns': 12,
 'updatedRange': "'VA02 Results'!A2:L194",
 'updatedRows': 193}

END